In [1]:
import pandas as pd
import uuid
from dataframe_sql import register_temp_table, query
import ast
import random
from tqdm import tqdm
import re

In [2]:
test_set = pd.read_json('wikisql_ann_test.json')

In [3]:
test_set.head()

,answer,question,id,header,rows,header_types
0,SELECT ` <col2> ` FROM ` table ` WHERE ` <col0...,<col0> Player : text <col1> No. : text <col2> ...,4e84108c351d47b5aedac962af6749cf,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
1,SELECT ` <col5> ` FROM ` table ` WHERE ` <col4...,<col0> Player : text <col1> No. : text <col2> ...,f6e1c2213a1d4cebbb1521da219c7aa0,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
2,SELECT ` <col5> ` FROM ` table ` WHERE ` <col4...,<col0> Player : text <col1> No. : text <col2> ...,1ce1252d4098425aaa9c38aeb5b79c69,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
3,SELECT COUNT ` <col5> ` FROM ` table ` WHERE `...,<col0> Player : text <col1> No. : text <col2> ...,33caff92db0442fa8870dee4ace70c4a,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
4,SELECT ` <col2> ` FROM ` table ` WHERE ` <col3...,<col0> No : real <col1> Date : text <col2> Rou...,acaa5428b2d541c7a7842de42e6fc77f,"['No', 'Date', 'Round', 'Circuit', 'Pole Posit...","[['1', '24 February', 'Qatar', 'Losail', 'Kevi...","['real', 'text', 'text', 'text', 'text', 'text..."


In [4]:
def register_table(query, data, row_id):
    tab_id = random.randint(0, 10000000)
    query = query.replace('` table `', f"table_{tab_id}")
    query = query.replace('` ','\'').replace(' `','\'')
    query = query.replace(' \'','\'')
    register_temp_table(data, f"table_{tab_id}")
    return query

In [5]:
answers = []
agg_operators = ['MAX', 'AVG', 'MIN', 'COUNT', 'SUM']
for idx, row in tqdm(test_set.iterrows()):
    answer = [[]]
    try:
        sql = row.answer
        question = row.question.split('</s>')[1]
        
        header = ast.literal_eval(row.header)
        header = [re.sub('[^A-Za-z0-9]+', '', col) for col in header]
        header_types = ast.literal_eval(row.header_types)
        rows = ast.literal_eval(row.rows)
        table = pd.DataFrame(rows, columns=header)
        for ix, column in enumerate(header):
            if f"` <col{ix}> `" in sql:
                if ' ' in column:
                    sql = sql.replace(f"` <col{ix}> `", f'\'{column}\'')
                else:
                    sql = sql.replace(f"` <col{ix}> `", column)
        for agg_operator in agg_operators:
            if agg_operator in sql:
                sql = sql.replace(agg_operator+' ', f' {agg_operator}(')
                sql = sql.replace(' FROM', f') FROM')
        for idx_, column_type in enumerate(header_types):
            if column_type == 'real':
                table[header[idx_]] = table[header[idx_]].astype(float)
                
        sql_f = register_table(query=sql, data=table, row_id=idx)
        for match in  re.findall('=\'[0-9.]+\'', sql_f):
            match_new = match.replace('\'','')
            sql_f = sql_f.replace(match, match_new)
        
        for op in  ['>=','=<','>','<']:
            if op in  sql_f:
                chars = '' 
                for char in sql_f.split(op)[1][1:]:
                    if char.isdigit() or char == '.':
                        chars += char 
                    else:
                        break
                sql_f = sql_f.replace(f"\'{chars}\'", chars)
        answers.append(query(sql_f).values)
    except Exception as e:
        answers.append(answer)
        continue

15878it [27:08,  9.75it/s]


In [14]:
test_set['answer_rows'] = answers

In [32]:
len(answers)

15878

In [53]:
counter = 0
len_g = 0
data_set = []
for ix,row in test_set.iterrows():
    try:
        if len(answers[ix][0]):
            counter += 1
            if len(row['answer_rows'][0]) >= 1 and row['answer_rows'][0][0] == row['answer_rows'][0][0]:
                len_g +=1
                #print((row['answer_rows']))
                data_set.append(dict(row))
    except Exception as e:
        pass
len(data_set)

5937

In [52]:
pd.DataFrame(data_set).shape#.to_csv('wikisql_ann_test_tapas_clean.csv', index=False,encoding='utf-8-sig')

(5937, 7)

In [31]:
ast.literal_eval('[[nan]]')

ValueError: malformed node or string: <_ast.Name object at 0x7fdfe5497d90>